# __Segmentacion de Comercio Online__

## Importar librerias

In [1]:
# librerias para el dataframe y la visualizacion
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

## Cargar los datos

In [3]:
retail_df = pd.read_csv('./OnlineRetail.csv', sep=',', encoding='ISO-8859-1', header=0)
retail_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


In [4]:
# tamano del dataset
retail_df.shape

(541909, 8)

> El dataset presenta 541.909 registros y solo 8 columnas.

In [5]:
# tipo de dato de cada variable
retail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


> 5 de las 8 variables son de tipo object. Hay que tenerlo en cuenta. 

## Acerca de los datos

Este es un conjunto de datos transaccionales que contiene todas las transacciones realizadas entre el 01/12/2010 y el 09/12/2011 para una tienda online sin establecimientos físicos con sede en el Reino Unido. La empresa vende principalmente regalos únicos para cualquier ocasión. Muchos de sus clientes son mayoristas.

Las variables que contienen los datos son:

- _InvoiceNo_: Numero de factura. Es un numero entero de 6 digitos asignado de forma unica a cada transaccion. si este codigo empieza con la letra "c", indica una cancelacion.

- _StockCode_: Codigo de stock del producto. Es un numero de 5 digitos asignado de forma unica a cada producto.

- _Description_: Nombre del producto.

- _Quantity_: Cantidades de cada producto por transaccion.

- _InvoiceDate_: Fecha de factura. Marca el dia y la hora en que se genero cada transaccion.

- _UnitPrice_: Precio del producto por unidad.

- _CustomerID_: Numero integral de 5 digitos asignado de forma unica a cada cliente.

- _Country_: Nombre del pais donde reside cada cliente.

## Resumen estadistico de las variables numericas

In [6]:
retail_df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


__Sobre Cantidad__

> Hay 541.909 datos, valor que coincide con la cantidad de registros. Esto indica que la variable no presenta valores faltantes.

> Presenta una media de 9.5 unidades por transaccion.

> El valor de std es muy alto (218.08) lo que indica que los datos estan muy dispersos probablemente por la presencia de valores atipicos. La distribucion no es normal.

> 25% de las transacciones tienen 1 o 0 unidades.

> 50% de las transacciones tienen 3 o menos unidades.

> 75% tienen 10 o menos unidades.

> El valor de min es negativo lo que posiblemente representa devoluciones.

__Sobre Precio Unitario__

> Preenta la misma cantidad de registros lo que nos indica que la variable no presenta valores faltantes.